In [ ]:
!pip install nltk
!pip install openpyxl
import re
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import os
nltk.download('vader_lexicon')

In [ ]:
path = os.getcwd()
print(path)

In [ ]:
# Read in the csv file
df = pd.read_excel(r"TrueAndFakeWITHALL.xlsx")
# adding an row_id field to the dataframe, which will be useful for joining later
df["row_id"] = df.index + 1
#print first 10 rows 
print (df.head(10))

In [ ]:
#create a new data frame with "id" and "comment" fields
df_subset = df[['row_id', 'Response']].copy()
#data clean-up
#remove all non-aphabet characters
df_subset['Response'] = df_subset['Response'].str.replace("[^a-zA-Z#]", " ")
#covert to lower-case
df_subset['Response'] = df_subset['Response'].str.casefold()

In [ ]:
# set up empty dataframe for staging output
df1=pd.DataFrame()
df1['row_id']=['99999999999']
df1['sentiment_type']='NA999NA'
df1['sentiment_score']=0


In [ ]:
print('Processing sentiment analysis...')

sid = SentimentIntensityAnalyzer()
t_df = df1
for index, row in df_subset.iterrows():
    scores = sid.polarity_scores(row[1])
    for key, value in scores.items():
        temp = [key,value,row[0]]
        df1['row_id']=row[0]
        df1['sentiment_type']=key
        df1['sentiment_score']=value
        t_df=t_df.append(df1)
#remove dummy row with row_id = 99999999999
t_df_cleaned = t_df[t_df.row_id != '99999999999']
#remove duplicates if any exist
t_df_cleaned = t_df_cleaned.drop_duplicates()
# only keep rows where sentiment_type = compound
t_df_cleaned = t_df[t_df.sentiment_type == 'compound']
print(t_df_cleaned.head(10))

In [ ]:
#merge dataframes
df_output = pd.merge(df, t_df_cleaned, on='row_id', how='inner')
print(df_output.head(10))

In [ ]:
df_output[["sentiment_score"]].describe()


In [ ]:
#generate mean of sentiment_score by period
dfg = df_output.groupby(['Period'])['sentiment_score'].mean()
#create a bar plot
dfg.plot(kind='bar', title='Sentiment Score', ylabel='Mean Sentiment Score',
         xlabel='Period', figsize=(6, 5))

In [ ]:
dfg = df_output.groupby(['News'])['sentiment_score'].mean()
dfg.plot(kind='bar', title='sentiment_score', ylabel='Mean Sentiment Score',
         xlabel='News', figsize=(6, 5))

In [ ]:
df_output.groupby(['News'])['sentiment_score'].describe()

In [ ]:
import seaborn as sns
#create seaborn boxplots by group
sns.boxplot(x='News', y='sentiment_score', notch = True,
            data=df_output, showfliers=False).set(title='Sentiment Score by News')
#modify axis labels
plt.xlabel('News')
plt.ylabel('Sentiment Score')
plt.xticks(rotation=90)